# Differential Privacy

> "You can't stop change any more than you can stop the sun from setting." ~ Shmi Skywalker

In this session we'll be learning about **Differential Privacy.** This is divided into two sections.

Section 1 will focus on **intuition**: 
* Intuition
    * What is differential privacy (DP)?
    * How does DP work?
        * How does adding noise protect privacy?
        * How much noise do we add?
        * What is the tradeoff?
    * What is the privacy budget?
        * Connection between privacy budget and risk
    * What is epsilon?
    
Section 2 will focus on DP in **our codebase**:
* DP in PySyft
    * How DP in PySyft is different than DP elsewhere
        * Adversarial
        * Individual
        * Automatic
    * Differential Privacy Tensors
        * PhiTensor
        * GammaTensor
        * Helper Classes:
            * LazyRepeatArray
            * DataSubjectArray
    * Ledgers and Privacy Budget Accounting
    * Sigma and noise addition

## Motivation 

In a previous session, we had discussed how difficult it is to protect people's privacy when working with or releasing data. We discussed the Netflix prize, where participants were de-anonymized with shocking accuracy. We mentioned problems caused by Data Linkages, and talked about the copy problem.

We also discussed potential solutions to these problems in the form of Privacy Enhancing Technologies (PETs). Differential Privacy is a one such PET which helps to protect Data Subjects by controlling the Output Privacy of computations.

We'll unpack this in more detail. But first, let's quickly standardize some terminology.

<img src="data:image/webp;base64,
...
IDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CgA=
" />

This is the standard `journey` of data in data science:
* Raw data is collected from lots of people like you and I, called **data subjects**  
* This raw data is often cleaned and preprocessed by **data owners** 
* The data owners then pass on these datasets to their data scientists, who then test their algorithms or workflows on the data to answer questions, draw useful conclusions, or build products 

## Section 1: Intuition

Put simply, `differential privacy` is a mathematical guarantee that if you run an algorithm against a dataset, the output is indistinguishable from running the same algorithm, against the same dataset without a specific `data subject` being included.

The result of such a guarantee is that it should not be possible to guess if a specific `data subject` is in a dataset. Therefore by extension the privacy of any given individual is protected because you cannot learn anything about someone if you cannot know with any certainty if they are present or absent from the dataset used in your algorithm.

### With or without you

First let's see what the real difference would be between a query with or without your data.

<img src="data:image/webp;base64,
...
" />

### With added noise

Now let's see how we can hide that difference by just adding noise so it looks similar.

<img src="data:image/webp;base64,
...
">

This means the `Data Scientist` still gets a reasonable and accurate answer of approximately `~25.7`, but can't be sure if you were in the dataset; and if you were, what impact your `age` had on the `average`.

So in a nutshell, `DP` makes it so that the Data Scientist doesn't work with just datasets, they work with datasets, plus enough noise to protect the privacy of the data subjects, while still giving useful results.

<img src="data:image/webp;base64,
...
REY+CjwveDp4bXBtZXRhPgoA"
/>

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### How Much Noise To Add? (in English)

Thus far, we've seen how differential privacy (DP) leads to the addition of a small amount of noise to protect the privacy of someone's data.

A reasonable next question to ask is ***how much noise do we add?***

Let's start with an easy, intuitive answer:

<img src="data:image/webp;base64,
...
" />

In the image above, we haven't added enough noise to properly protect the person's privacy. We need to add more!

<img src="data:image/webp;base64,
...
" />

This time, we went way overboard with adding noise. The Data Scientist is going to suffer a serious loss of accuracy in their calculations. We need to add less!

What this should hopefully convey to you is that there is a **tradeoff** between the amount of noise we add, and the corresponding privacy and utility of the results we get from our algorithm.

* If we add too much noise, the results are no longer accurate, rendering the algorithm useless
* If we don't add enough noise, the results are accurate, but we aren't able to protect anyone's privacy

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### How Much Noise to Add? (in Math)

DP is a mathematical guarantee, and so there is a mathematical answer to this question.
But before we can answer this question of how much noise to add, we'll need to understand a key insight: **datasets are distributions.**

Let's explore this idea with a simple example.

Imagine you had a dataset consisting of the numbers:  
`[1, 2, 3, 4, 5]`

An easy way you could convert this from a dataset into a distribution is if you iterated through every datapoint in the dataset, and asked, `what is the probability of number n being in this dataset?`.

The results could be shown as a histogram that looks like this:

<img src="data:image/webp;base64,
...
" />

Voila! Using this simple scheme, we've converted our dataset into a probability distribution.

You may ask: *"What was the point of that?"*  

Why did we need to convert our dataset into a distribution?  
Well, it turns out, there are lots of useful ways to compare two distributions!

Here are sixteen just for example:

<img src="data:image/webp;base64,
...
IDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CgA=
" />

This means we now have a well defined way to compare any two datasets, by simply converting them to distributions first, and then using any of the example methods shown above.

You may ask: *"But Ishan, why is it important to compare datasets?"*  

What does comparison have to do with figuring out how much noise we want to add?

Well, let's revisit the definition of `Differential Privacy`:  
The result of an algorithm is ***similar*** when a single person's data is removed from a dataset.

**Intuition:** As soon as we know how similar the results are, we immediately know how much noise is enough, and how much is too much. In addition, calculating this similarity is much easier if we treat our datasets as if they were distributions.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Introducing $\epsilon$, the privacy budget

This "similarity" or difference is captured in a parameter called $\epsilon$ (**epsilon**), which we often refer to as the **privacy budget.**

$\epsilon$ has different formulas (depending on which method we used to compare our datasets or distributions), but it is always a measure of how much your data (a data subject) affects the algorithms result.

<img src="imgs/dp_intro_epsilon.png">

$\epsilon$, or the `privacy budget`, is probably the most important idea in `Differential Privacy`, so it's worth taking some time to emphasize exactly what is is.

The privacy budget, $\epsilon$, is a measure of:
* How much your data stands out
    * Thus, it's also a measure of *privacy risk*; how likely your data is going to be identified
* How much your data affects the outcome of the query or algorithm
* How much noise is needed to hide your data's influence

**Note:** 
There is a more rigorous mathematical definition of epsilon (see resources below), but thinking about it in this way helps to build an intuition.

### The Effects of $\epsilon$

**The higher the $\epsilon$, the more your data affects the outcome of an algorithm.**

<img src="data:image/webp;base64,
...
OlJERj4KPC94OnhtcG1ldGE+CgA=
" />

For example:
* Imagine if we're trying to calculate the net worth of all the people living inside a single city, and we did this for every city in the United States. Warren Buffet would have a gigantic value of $\epsilon$, compared with nearly any every other person.

**The higher the $\epsilon$, the more noise needs to be added to protect your privacy.**

This should make intuitive sense:
* As we discussed above, the higher the epsilon, the more your data affects the outcome of an algorithm
* The more your data affects the outcome of an algorithm, the more noise you need to add to mask the effects of your data

<img src="data:image/webp;base64,
..." />

**In theory**, noise is a random value that depends on $\epsilon$.  
**In practice**, we obtain it by sampling from a distribution where $\epsilon$ is a parameter. 

Often, it's a normal (Gaussian) distribution where $\epsilon$ affects the standard deviation.
So the bigger the $\epsilon$, the wider the range of values the noise will likely be sampled from [1]:

<img src="data:image/webp;base64,
...
bXBtZXRhPgo=
" />

[1]: We know this because of the empirical (68-95-99.7) rule in statistics, which says that when you sample a normal distribution, approximately 68% of the values will be within one standard deviation of the mean, 95% will be within 2 standard deviations of the mean, and 99.7% will be within 3 deviations of the mean, and so forth.

This rule is also how the company Six Sigma got its name! They aim to get manufacturing processes error rates to a six sigma rate, which essentially means no more than 3.4 defects in every one million fabrications.

<img src="data:image/webp;base64,
...
dGE+Cg==
">

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

### Why does DP work?

I'd like to leave you with a intuitive, gut feeling as to why `Differential Privacy` (this method of adding `noise`) actually works to `protect` privacy.

The intuition is that when it comes to protecting privacy, your data "hides" in the background of other people's data. The more your data fits in with other people's data, the easier it is to blend in and hide in plain sight.

Put another way, it's much easier to spot the horse in the image on the left, than the image on the right.

<img src="data:image/webp;base64,
...
" />

<br>

Again, the best way to hide a data point, is to surround it with other, very similar data points! They'll be hard to tell apart, and it'll be like trying to look for a piece of **hay** in a very large, fluffy haystack:

<img src="data:image/webp;base64,
...
PC94OnhtcG1ldGE+CgA=" />

By adding noise, we're doing something very similar- we're making it hard to discern one data point from another; the same way how dumping all your hay in a haystack makes it hard to single out individual pieces of hay.

### Takeaways

Differential Privacy:
- Allows you to learn `high level statistics` and trends about things like:
    - How big is the haystack?
    - Am I running out of hay?
    - Is my hay on fire?
- But, `not` specific statistics about `individual data` like:
    - is the piece of hay in the center getting moldy?

This means you can:
- Answer questions such as `What causes cancer?` but not `Does Walter White have cancer?`
- Differential Privacy works by adding noise to the output of an algorithm to protect privacy
- $\epsilon$, the privacy budget, is an indicator of how likely it is that someone's data stands out and can be identified:
    - The higher the $\epsilon$, the more likely that person's data stands out and will be identified
    - $\epsilon$ can be calculated in many ways, but it's always an indicator of the risk of being identified and having your privacy violated

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Section 2: DP in PySyft

The goal of this section is to help you understand the major components of the Differential Privacy system in PySyft, as well as how PySyft's DP system is different than regular DP systems elsewhere.

Differential Privacy in `PySyft` is different in 3 crucial ways:

### 1. It's Adversarial

- There is a hard limit on how much any `Data Scientist` who interacts with a dataset can learn about it
- This limit is how much privacy budget ($\epsilon$) the `Data Owner` gives them

For example, say the Data Owner gives a Data Scientist 5 $\epsilon$ of privacy budget. Every time they want to get a result, it might cost them a few $\epsilon$, and the system would deduct this from their available balance.

### 2. It's Individual

- We track the privacy risk for `Data Subject` in a dataset (Individual Differential Privacy)  
- This is useful because by knowing exactly how much each person's privacy is at risk, we can choose to remove only their data from a computation if it gets too close to violating their privacy
- This gives Data Scientists more "mileage", since less noise on the rest of the data set is better than the extreme level of noise needed to hide that high risk person if we didn't remove them
- If a given algorithm violates any individual person's privacy too much, the Data Scientist gets penalized by having more $\epsilon$ deducted

This is in contrast to regular Differential Privacy, where an entire dataset is characterized by a single $\epsilon$. This is because regular DP only considers the privacy risk of the individual whose data is most at risk. There is usually good reason for this, because as soon as one person's privacy is violated, you have a `data leak` and your infrastructure is by definition not secure.

### 3. It's Automatic

- By keeping a log of all DP Queries on a ledger we can know if the noise added to a result for a given `Data Scientist` is within their budget
- If the query is within privacy budget ($\epsilon$), a `Data Scientist` can automatically `get` the result without waiting for approval (Automatic Differential Privacy)  

<br>

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## PySyft's DP Scenario

Let's say we have a Data Scientist. This person was given $\epsilon$ = 10 of privacy budget to work with.
The dataset they're using has 3 people's data in it: Rob, Bob, and Job.

<img src="data:image/webp;base64,
...
" />

Let's say Rob and Bob's data is both `4`, and Job's data is `6`.

Although this dataset is small, we can intuitively tell that Job's data sticks out more than Rob's and Bob's. Thus, we expect his privacy to be more at risk, and any epsilon for Job to be higher.

The fact that PySyft is able to reason this way, and deduce privacy risk for each person in the dataset; is an example of it's **individual** nature at work.

Let's say the Data Scientist wants to do a `sum()` operation:

<img src="data:image/webp;base64,
...
L3JkZjpSREY+CjwveDp4bXBtZXRhPgoA
" />

## Components

Now that we've seen the entire DP process, end-to-end in PySyft, let's deep dive into the key components:
- Tensors
- Ledgers
- Publishing

### DP Tensors in PySyft

The first (and arguably most used) component in our DP system are our custom Tensor classes.

The biggest difference between our Tensor classes and other common `tensor` or `array` implementations such as `torch.tensor()` or `np.array()`, is that our Tensors not only store the regular numerical data, but also store sidecar *metadata.*

This is a regular `torch` tensor:

In [4]:
import torch
torch.tensor([1, 2, 3, 4])

tensor([1, 2, 3, 4])

This is a regular `numpy` array:

In [5]:
import numpy as np
np.array([1, 2, 3, 4])

array([1, 2, 3, 4])

As you can see, they both store the same data: `[1, 2, 3, 4]` and allow you to easily do math operations.

In [6]:
torch.Tensor([1, 2, 3, 4]) + 2

tensor([3., 4., 5., 6.])

In [7]:
np.array([1, 2, 3, 4]) + 2

array([3, 4, 5, 6])

The `tensors` in `syft` also provide the ability to store data:

In [8]:
import syft as sy
sy.Tensor([1, 2, 3, 4])

Tensor(child=[1 2 3 4])

<br>
... as well as do arithmetic on them:

In [9]:
sy.Tensor([1, 2, 3, 4]) + 2

Tensor(child=[3 4 5 6])

But with the addition of metadata, our `tensor` objects can help us figure out how much noise we should add to protect privacy.

Specifically, we track 3 kinds of metadata:
- a theoretical `lower bound` on the data held in the tensor
- a theoretical `upper bound` on the data held in the tensor
- the `data subjects`, or the people whose data is stored in the tensor (and thus whose privacy we're protecting)

This metadata is provided by calling `.private()` on a `sy.Tensor` like so:

In [10]:
sy.Tensor([1, 2, 3, 4]).private(min_val=0, max_val=5, data_subjects=["Ishan"])

Tensor(child=PhiTensor(child=[1 2 3 4], min_vals=<lazyrepeatarray data: [0] -> shape: (4,)>, max_vals=<lazyrepeatarray data: [5] -> shape: (4,)>))

By knowing the `lower` and `upper` bounds of the theoretical distribution, we can calculate how much noise is too much and too little.

### Too much noise

For instance, if I `sum` 10 data points in the range  of [0, 10] `(lower_bound=0, upper_bound=10)`, and I then added noise of `300`, that noise would skew the results so heavily that the signal of the true `sum` would be mostly lost.

In [13]:
signal = sum([1, 2, 3, 4, 5, 3, 4, 2, 1, 8])
print("Signal =", signal)
noise = 300
print("Noise =", noise)
result = signal + noise
print("Final result after DP =", result)

Signal = 33
Noise = 300
Final result after DP = 333


### Not enough noise

On the other hand, if I `sum` 10 data points in the range of [1,000,000 , 10,000,000] `(lower_bound=1_000_000, upper_bound=10_000_000)`, and I add a noise value of `50`, that's probably too little noise to meaningfully protect anyone's privacy.

In [14]:
signal = sum([1e6, 2e6, 3e6, 4e6, 5e6, 3e6, 4e6, 2e6, 1e6, 8e6])
print("Signal =", signal)
noise = 50
print("Noise =", noise)
result = signal + noise
print("Final result after DP =", result)

Signal = 33000000.0
Noise = 50
Final result after DP = 33000050.0


For those of you who've done signal processing before, this might look similar to the idea of **signal-to-noise.**

### Data Subjects

Meanwhile, **knowing the data subjects helps us figure out whose privacy we're protecting.**

This is tremendously useful when it comes to things like Linkage Attacks (refer to Session 01 - the Netflix Challenge example). If we're able to keep track of which data belongs to which person, we could quantify in strict mathematical terms ($\epsilon$) how much that individual's privacy is at risk.

Based on the number of Data Subjects in a Tensor, we can split our DP Tensors into two distinct types:
- PhiTensor: `1 unique` data subject `per data point` in the Tensor
- GammaTensor: `2 or more unique` data subjects `per data point` in the Tensor

Let's try playing around with them!

In [13]:
# Let's say we have some data:
data = np.random.randint(low=1, high=7, size=(5,5))
data

array([[6, 4, 4, 6, 5],
       [5, 4, 3, 2, 2],
       [1, 6, 2, 6, 5],
       [1, 5, 6, 6, 1],
       [6, 4, 5, 5, 1]])

In [14]:
# We put it in our Syft Tensor:
tensor = sy.Tensor(data)
tensor

Tensor(child=[[6 4 4 6 5]
 [5 4 3 2 2]
 [1 6 2 6 5]
 [1 5 6 6 1]
 [6 4 5 5 1]])

In [15]:
# We then annotate our Tensor with Metadata, thus creating a PhiTensor underneath the hood:
private_tensor = tensor.private(min_val=0, max_val=10, data_subjects="Ishan")
private_tensor

Tensor(child=PhiTensor(child=[[6 4 4 6 5]
 [5 4 3 2 2]
 [1 6 2 6 5]
 [1 5 6 6 1]
 [6 4 5 5 1]], min_vals=<lazyrepeatarray data: [0] -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: [10] -> shape: (5, 5)>))

In [11]:
private_tensor.child

PhiTensor(child=[[6 5 1 3 3]
 [4 1 1 3 6]
 [4 1 3 3 6]
 [3 1 4 3 1]
 [2 4 2 3 5]], min_vals=<lazyrepeatarray data: [0] -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: [10] -> shape: (5, 5)>)

This is the first of our DP tensors- called a **PhiTensor**.
A PhiTensor only has one unique data subject per value in the Tensor. You can see this below:

In [15]:
# This array has the same shape as our NumPy data- so there's a 1:1 correspondence.
private_tensor.child.data_subjects

array([[DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}],
       [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}],
       [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}],
       [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}],
       [DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}, DataSubjectArray: {'Ishan'},
        DataSubjectArray: {'Ishan'}]], dtype=object)

If you add two PhiTensors belonging to different people, you would get a **GammaTensor**.
This is shown below:

In [ ]:
second_tensor = sy.Tensor(data).private(min_val=0, max_val=10, data_subjects="Carl")

gamma_tensor = private_tensor + second_tensor
gamma_tensor.child

And you can see the data subjects in this array:

In [ ]:
gamma_tensor.child.data_subjects

There's one important thing to note here. Once we have a GammaTensor, we need to be extra careful since we are combining private data belonging to two different people in a single Tensor.

We keep track of every operation that occurs with a GammaTensor in a dictionary called `source`. You can think of this as a tree keeping track of which input tensors combined to give which output tensors.

The key in this dictionary is an integer (unique to every GammaTensor) which maps to the corresponding GammaTensor.

We can see below how `gamma_tensor`, which was created by adding `private_tensor` and `second_tensor`, has both of those tensors in its `source` tree:

In [ ]:
gamma_tensor.child.sources

GammaTensors also tracks which operation created it in `self.func_str`:

In [ ]:
gamma_tensor.child.func_str

Notes:

- You might imagine that as we do more and more operations, this source tree can become gigantic. You're correct.
- This is why we keep track of PhiTensors separately. Many data science workflows can and are being done with data belonging to just a single individual, in which case we don't need to take the precautions associated with GammaTensor.

### DP Ledger in PySyft

We mentioned earlier that a unique part of PySyft is that we keep track of every data subject's $\epsilon$. This is done in our `DataSubjectLedger` class.

One small nuance about how we store epsilon: it involves a cache.

### Publishing in PySyft